[github link](https://github.com/ilyakava/sumproduct)

## Interface
```python
g = FactorGraph()

x1 = Variable('x1', 2) # init a variable with 2 states
x2 = Variable('x2', 3) # init a variable with 3 states

f12 = Factor('f12', np.array([
  [0.8,0.2],
  [0.2,0.8],
  [0.5,0.5]
])) # create a factor, node potential for p(x1 | x2)

# connect the parents to their children
g.add(f12)
g.append('f12', x2) # order must be the same as dimensions in factor potential!
g.append('f12', x1) # note: f12 potential's shape is (3,2), i.e. (x2,x1)

g.compute_marginals() # -> [0.0]

g.nodes['x1'].marginal() # -> array([0.5, 0.5])
```

In [4]:
import numpy as np

In [27]:
isinstance(np.array([]), np.ndarray)

True

In [135]:
class Message:
    def __init__(self, origin, dest, values):
        self.origin = origin
        self.dest = dest
        self.values = values

In [295]:
class Node:
    def __init__(self, name):
        self.name = name
        self.neighbors = []
        self.messages = []
    
    def __repr__(self):
        return self.name
    
    def add_neighbor(self, factor):
        self.neighbors.append(factor)
    
    def send_message(self):
        # when node is requested to send message and list of incoming messages is empty,
        # this means that node is leaf node and most use init message
        if len(self.messages) == 0:
            message = np.ones(len(self.states)) if isinstance(self, Variable) else self.values
            for node in self.neighbors:
                node.messages.append(Message(self, node, message))
                print(f'Node {self.name} sends message to node {node.name}')
            return self.neighbors # return all neighbors
        else: # not a leaf
            in_values= [message.values for message in self.messages]
            in_nodes = [message.origin for message in self.messages]
            out_nodes = [node for node in self.neighbors if node not in in_nodes]
            for node in out_nodes:
                print(f'Node {self.name} sends message to node {node.name}')
                if isinstance(self, Variable):
                    # from variable to factor: product of all in messages:
                    message = np.prod(in_values, axis=0)
                    node.messages.append(Message(self, node, message))
                else: # factor node
                    pass
            return out_nodes

In [296]:
one1 = 3*np.ones(3)
one2 = 2*np.ones(3)
np.prod([one1, one2], axis=0)

array([6., 6., 6.])

In [297]:
class Variable(Node):
    def __init__(self, name, n_states):
        super().__init__(name)
        self.states = list(range(n_states))    

In [298]:
class Factor(Node):
    def __init__(self, name, array):
        super().__init__(name)
        assert isinstance(array, np.ndarray), f'´array has to be a numpy ndarray, not {type(array)}'
        self.values = array
        self.shape  = self.values.shape


In [299]:
class FactorGraph:
    def __init__(self):
        self.factors = []
        self.vars = []
    
    def add(self, factor):
        """Add a factor to the graph"""
        assert factor not in self.factors, f'Factor {factor.name} already defined'
        self.factors.append(factor)
    
    def append(self, factor, variable):
        """Add a variable to factor"""
        assert factor in self.factors, f'Factor {factor.name} not yet defined'
        # TODO: check for correct size
        if variable not in self.vars:
            self.vars.append(variable)
        factor.add_neighbor(variable)
        variable.add_neighbor(factor)

    def leafs(self):
        leafs = set()
        for node in self.vars + self.factors:
            # a leaf is a node that is missing one incoming message
            if len(node.neighbors) == 1:
                leafs.add(node)
        return leafs
    
    def compute_marginals(self):
        leafs = self.leafs()
        # 1. pick (arbitrary) root node
        #root = leafs.pop()
        root = self.factors[-1] # force uses of  as root - remove later
        leafs.remove(root)
        # 2. Propagate messages from leaves to root
        fringe = leafs.copy()
        for node in fringe:
            next_nodes = node.send_message()
            fringe = fringe.union(next_nodes)
            fringe.remove(node)
                    

In [300]:
## test
g = FactorGraph()

x1 = Variable('x1', 2) # init a variable with 2 states
x2 = Variable('x2', 3) # init a variable with 3 states

f12 = Factor('f12', np.array([
  [0.8,0.2],
  [0.2,0.8],
  [0.5,0.5]
])) # create a factor, node potential for p(x1 | x2)

# connect the parents to their children
g.add(f12)
g.append(f12, x2) # order must be the same as dimensions in factor potential!
g.append(f12, x1) # note: f12 potential's shape is (3,2), i.e. (x2,x1)

#g.compute_marginals() # -> [0.0]

#g.nodes['x1'].marginal() # -> array([0.5, 0.5])



In [301]:
g.leafs()

{x1, x2}

In [302]:
## second test
g = FactorGraph()

A = Variable('A', 2)
B = Variable('B', 2)
C = Variable('C', 2)
D = Variable('D', 2)

f1 = Factor('f1', np.array([
    [10, 1],
    [1, 10]
]))

f2 = Factor('f2', np.array([
    [1, 10],
    [10, 1]
]))

f3 = Factor('f3', np.array([
    [10, 1],
    [1, 10]
]))

f4 = Factor('f4', np.array(
    [10, 1]
))

g.add(f1)
g.append(f1, A)
g.append(f1, B)

g.add(f2)
g.append(f2, B)
g.append(f2, C)

g.add(f3)
g.append(f3, B)
g.append(f3, D)

g.add(f4)
g.append(f4, C)

In [303]:
g.factors

[f1, f2, f3, f4]

In [304]:
g.compute_marginals()

Node D sends message to node f3
Node A sends message to node f1


In [279]:
f3.messages[0].array

array([1., 1.])

In [306]:
a = set(); a.add(1)
b = set(); b.add(1)
c = set([1])

In [315]:
a  = a.union([2])

In [317]:
a.difference(b)

{2}